<a href="https://colab.research.google.com/github/hassanSattariNia/FederatedLearning/blob/main/RESNET_18_distributed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

# handle device work on GPU or CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
list_accuracies = []


# define basic Block
class BasicBlock(nn.Module):
    expansion = 1
    def __init__(self, in_channels, out_channels, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != self.expansion * out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, self.expansion * out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion * out_channels)
            )
    def forward(self, x):
        out = torch.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = torch.relu(out)
        return out

# class partitions
class Partition1(nn.Module):
    def __init__(self):
        super(Partition1, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)

    def forward(self, x):
        out = torch.relu(self.bn1(self.conv1(x)))
        return out

class Partition2(nn.Module):
    def __init__(self, in_channels):
        super(Partition2, self).__init__()
        self.layer1 = self._make_layer(BasicBlock, in_channels, 64, 2, stride=1)
        self.layer2 = self._make_layer(BasicBlock, 64, 128, 2, stride=2)

    def _make_layer(self, block, in_channels, out_channels, num_blocks, stride):
        layers = []
        layers.append(block(in_channels, out_channels, stride))
        in_channels = out_channels * block.expansion
        for _ in range(1, num_blocks):
            layers.append(block(in_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        return out

class Partition3(nn.Module):
    def __init__(self, in_channels):
        super(Partition3, self).__init__()
        self.layer3 = self._make_layer(BasicBlock, in_channels, 256, 2, stride=2)
        self.layer4 = self._make_layer(BasicBlock, 256, 512, 2, stride=2)

    def _make_layer(self, block, in_channels, out_channels, num_blocks, stride):
        layers = []
        layers.append(block(in_channels, out_channels, stride))
        in_channels = out_channels * block.expansion
        for _ in range(1, num_blocks):
            layers.append(block(in_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.layer3(x)
        out = self.layer4(out)
        return out

class Partition4(nn.Module):
    def __init__(self, in_channels, num_classes=10):
        super(Partition4, self).__init__()
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(in_channels, num_classes)

    def forward(self, x):
        out = self.avgpool(x)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

#create every partition
partition1 = Partition1().to(device)
partition2 = Partition2(64).to(device)
partition3 = Partition3(128).to(device)
partition4 = Partition4(512).to(device)


transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False, num_workers=2)

criterion = nn.CrossEntropyLoss()
optimizer =optimizer = optim.Adam(
    list(partition1.parameters()) +
    list(partition2.parameters()) +
    list(partition3.parameters()) +
    list(partition4.parameters()),
    lr=0.001  )
#  optim.SGD(list(partition1.parameters()) + list(partition2.parameters()) +
#                       list(partition3.parameters()) + list(partition4.parameters()), lr=0.01, momentum=0.9)

def calculate_accuracy():
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data[0].to(device), data[1].to(device)
            output_layer1 = partition1(images)
            output_layer2 = partition2(output_layer1)
            output_layer3 = partition3(output_layer2)
            output_layer4 = partition4(output_layer3)
            _, predicted = torch.max(output_layer4, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total

num_epochs = 5
for epoch in range(num_epochs):
    running_loss = 0.0
    print(f"Epoch {epoch + 1}/{num_epochs}")

    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)

        # Forward pass بدون .detach()
        output_layer1 = partition1(inputs)
        output_layer2 = partition2(output_layer1)
        output_layer3 = partition3(output_layer2)
        output_layer4 = partition4(output_layer3)

        # calculate Loss
        loss = criterion(output_layer4, labels)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:
            print(f"Batch {i + 1}, Loss: {running_loss / 100:.4f}")
            running_loss = 0.0

    accuracy = calculate_accuracy()
    print(f"Epoch {epoch + 1} finished with accuracy: {accuracy:.2f}%")
    list_accuracies.append(accuracy)
print('end train')


filename = "multi_part_resnet18.txt"
# Open the file in write mode
with open(filename, "w") as file:
  file.write(f'accuracies: {list_accuracies}')



100%|██████████| 170M/170M [00:03<00:00, 55.0MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
Epoch 1/5


KeyboardInterrupt: 